# LSIによる次元圧縮の動作確認

ストーリー: https://www.pivotaltracker.com/story/show/149341617

参考文献:
- https://abicky.net/2012/03/24/211818/
- http://scikit-learn.org/stable/modules/decomposition.html#truncated-singular-value-decomposition-and-latent-semantic-analysis

Gemsimを使うほうがもしかしたらいいかもしれない
- http://blog.yuku-t.com/entry/20110623/1308810518

今回は、sckit-learnのTruncatedSVDを用いて次元圧縮を行ってみる

In [1]:
import sys
import os

os.getcwd()

'/Users/shwld/project/mofmof/donusagi-bot/learning/prototype'

In [2]:
learning_dir = os.path.abspath("../")
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [3]:
from learning.core.nlang import Nlang
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## データの準備

In [4]:
df = pd.read_csv('./fixtures/learning_training_messages/toyotsu_human.csv')
sentences = np.array(df['question'])
separated_sentences = Nlang.batch_split(sentences)
separated_sentences

['住宅 ローン 法人 提携 銀行 トーメン 社員 申請 内容 修正 する 差戻し 願 する',
 '住宅 ローン 法人 提携 銀行 トーメン 社員 当社 提携 する 銀行 名 優遇 条件 連絡 窓口 等 知る',
 '在職 証明 書 欲しい',
 '住宅 ローン 手続き 法人 提携 銀行 トーメン 社員 不動産 法人 割引 紹介 カード 欲しい',
 '財形 貯蓄 財形 住宅 新規 募集 時期 いる',
 '財形 貯蓄 財形 住宅 金額 変更 随時 できる',
 '財形 貯蓄 財形 住宅 払戻し 願 する',
 '社宅 どこ 空き 状況',
 '出張 者 金町 寮 宿泊 する どう する いい',
 '引越 先 決まる ない どう する いい',
 '港 区 勤務 する 在勤 証明 書 発行 する ほしい',
 '人事 異動 出向 戻る 伴う ＰＣ アドレス 設定 どう やる',
 '育児 短時間 勤務 申請 書 いつ 提出 する',
 '育児 短時間 勤務 申請 書 曜日 別 申請 できる',
 '育児 短時間 勤務 申請 どこ 提出 する よい',
 '財形 貯蓄 財形 住宅 一部 解約 する 申込 振込 どれ かかる',
 '現在 借上げ 寮 いる 子供 大学 入学 する 上京 する 一緒 住める 寮 変更 する',
 'コーポレート カード 持つ 解る ない なる 調べる ほしい',
 '芝浦 アイランド 付近 住む 都 バス 利用 する 通勤 費 申請 する よい',
 '通勤 費 申請 最寄駅 バス 走行 距離 １ ｋｍ １ ｋｍ 未満 支給 なる なぜ',
 '東京 本社 近い バス 通勤 申請 する よい',
 '通勤 費 経済 的 合理 的 ルート',
 '通勤 費 月額 １０ 万 円 超える ない 新幹線 通勤 認める',
 '海外 送付 制度 利用 する 現地 発生 関税 費用 等 立替 精算',
 '現地 住所 決まる ない 引越 手配 進める られる',
 '予防 接種 料金 負担 先 どこ 勘定 科目',
 '休暇 申請 申請 内容 変更 する 申請 する いい',
 '在中 健康 診断 予約 変更 する どう する いい',
 '健保 健 診る 半年 受診 する 渡航 健 診る 代用 できる',
 '駐在 予定 中国 指定 健 診る 

## TfidfVectorizerを用いてベクトルを作る

In [5]:
vectorizer = TfidfVectorizer(use_idf=True, token_pattern=u'(?u)\\b\\w+\\b')
features = vectorizer.fit_transform(separated_sentences)
features.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

## TruncatedSVDで次元圧縮する

In [6]:
lsa = TruncatedSVD()
lsa_features = lsa.fit_transform(features)
normalizer = Normalizer(copy=False)
lsa_features = normalizer.fit_transform(lsa_features)
lsa_features

array([[ 0.82737512,  0.56164972],
       [ 0.6865807 ,  0.7270536 ],
       [ 0.77778737,  0.62852749],
       ..., 
       [ 0.725567  ,  0.68815153],
       [ 0.32687893,  0.94506622],
       [ 0.34623577,  0.93814753]])

## 検索する単語も同様に処理する

In [7]:
target_sentences = np.array(['保険証失くした'])
target_separated_sentences = Nlang.batch_split(target_sentences)
target_separated_sentences

['保険証 失 くい']

In [8]:
target_features = vectorizer.transform(target_separated_sentences)
lsa_target_features = lsa.transform(target_features)
lsa_target_features = normalizer.transform(lsa_target_features)
lsa_target_features

array([[ 0.39647295,  0.9180464 ]])

In [9]:
# LSI無し
print(features.shape)
print(target_features.shape)

# LSIあり
print(lsa_features.shape)
print(lsa_target_features.shape)

(584, 1097)
(1, 1097)
(584, 2)
(1, 2)


## コサイン類似検索をかけてみる(LSIなし)

In [10]:
similarities = cosine_similarity(features, target_features)
similarities = similarities.flatten()
zipped_data = zip(similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))

[(0.58960630289081473, '健康 保険証 紛失 する', '健康保険証を紛失した'),
 (0.58921785745677258, '保険証 盗む れる', '保険証が盗まれた'),
 (0.57929402755271386,
  '豊田通商 健保 任意 継続 する 保険証 いつ 届く 届く 保険証',
  '豊田通商健保を任意継続したが、保険証はいつ届くの？\u3000届くまでの保険証は？'),
 (0.48583404779265327, '保険証 印字 薄い なる', '保険証の印字が薄くなった'),
 (0.40529716736761873,
  'すぐ 病院 行く 保険証 忘れる どう する いい',
  'すぐに病院へ行きたいが保険証を忘れた、どうすればいいの？'),
 (0.0,
  '住宅 ローン 法人 提携 銀行 トーメン 社員 申請 内容 修正 する 差戻し 願 する',
  '住宅ローン（法人提携銀行・旧トーメン社員）申請内容を修正したいので、差戻しをお願したい'),
 (0.0,
  '住宅 ローン 法人 提携 銀行 トーメン 社員 当社 提携 する 銀行 名 優遇 条件 連絡 窓口 等 知る',
  '住宅ローン（法人提携銀行・旧トーメン社員）の当社と提携している銀行名、優遇条件、連絡窓口等を知りたい'),
 (0.0, '在職 証明 書 欲しい', '在職証明書が欲しい'),
 (0.0,
  '住宅 ローン 手続き 法人 提携 銀行 トーメン 社員 不動産 法人 割引 紹介 カード 欲しい',
  '住宅ローン手続き（法人提携銀行・旧トーメン社員）の不動産法人割引の紹介カードが欲しい。'),
 (0.0, '財形 貯蓄 財形 住宅 新規 募集 時期 いる', '財形貯蓄・財形住宅の新規募集時期はいつですか？'),
 (0.0, '財形 貯蓄 財形 住宅 金額 変更 随時 できる', '財形貯蓄・財形住宅の金額の変更は随時できますか？'),
 (0.0, '財形 貯蓄 財形 住宅 払戻し 願 する', '財形貯蓄・財形住宅の払戻しをお願したい。'),
 (0.0, '社宅 どこ 空き 状況', '社宅はどこにありますか？空き状況は？'),
 (0.0, '出張 者 金町 寮 宿泊 する どう する いい', '出

## コサイン類似検索をかけてみる(LSIあり)

In [11]:
lsa_similarities = cosine_similarity(lsa_features, lsa_target_features)
lsa_similarities = lsa_similarities.flatten()
zipped_data = zip(lsa_similarities, separated_sentences, np.array(df['question']))
list(sorted(zipped_data, key=lambda x: x[0], reverse=True))

[(0.99997317426298982, '雇用 保険 保険 者 証 コピー ほしい', '雇用保険被保険者証のコピーがほしい'),
 (0.99996700863759436, '豊田通商 健保 料 いくら かかる', '豊田通商の健保料はいくらかかるの？'),
 (0.99996630899893846, '引っ越す 手続 きき 必要', '引っ越したが、どんな手続ききが必要か'),
 (0.99992410657894149,
  '退職 後 独立 自営 する 雇用 保険 失業 保険 受給 できる',
  '退職後は独立（自営）するが雇用保険（失業保険）受給できるの？'),
 (0.9998764872396011,
  '退職 後 雇用 保険 失業 給付 手続き きする すぐ 支給 する れる',
  '退職後、雇用保険の失業給付手続ききすれば、すぐに支給されるの？'),
 (0.99986904645115326,
  '地方 税 住民 税 退職 年 支払 いく 翌年 役所 納付 請求 届く なぜ',
  '地方税（住民税）を退職年に支払いったはずなのに、翌年も役所より納付請求が届いたのはなぜ？'),
 (0.99983183717866975, 'ＯＢ 会 ＯＢ 会 退職 後 加入 できる', 'OB会（OB会）は、退職後でも加入できるか？'),
 (0.99979472288705329, '年金 手帳 会社', '年金手帳って会社にあるの？'),
 (0.99952227970257501,
  '退職 後 賞与 所得 税 健保 介護 保険 厚生 年金 雇用 保険 控除 する れる',
  '退職後の賞与について、所得税、健保、介護保険、厚生年金、雇用保険は控除されるの？'),
 (0.99936253440803724, '豊田通商 健保 継続 できる', '豊田通商の健保は継続できるの？'),
 (0.9993308372217039, '介護 休暇 どれ', '介護休暇は、どれくらいある？'),
 (0.99929359900256354, '扶養 家族 減る', '扶養家族が減りました'),
 (0.99927022040770042,
  '給与 天引き する 保険 がん 生命 保険 損害 保険 自動車 保険 手続 きき',
  '給与